In [2]:
%load_ext autoreload
%autoreload 2

In [105]:
import json

In [204]:
import qat

from qat.utils.pydantic import WarnOnExtraFieldsModel

from pydantic import ConfigDict, Field, model_validator, field_serializer, ValidationInfo, field_validator
from typing import Dict, List, Tuple, Union

import numpy as np

In [205]:
class HasId(WarnOnExtraFieldsModel):
    id: str

In [206]:
class At(HasId):
    x: int

In [238]:
class Bt(HasId):
    x: int
    As: Dict[str, At]

    @field_serializer('As')
    def serialize_Xs(self, Xs):
        return list(Xs.keys())

In [239]:
class Ct(HasId):
    x: int
    Bs: Dict[str, Bt] 
    As: Dict[str, At]

    @field_serializer('As', 'Bs')
    def serialize_Xs(self, Xs):
        return list(Xs.keys())

In [240]:
class Dt(HasId):
    x: int
    Cs: Dict[str, Ct]

    @field_serializer('As', 'Bs')
    def serialize_Xs(self, Xs):
        return list(Xs.keys())

This model is a DAG which means it can be serialised in layers

In [287]:
def hydrate(lst, context, key):
        d = {a.id: a for a in context}
        for ob in lst:
            if isinstance(ob, dict):
                newlst = {x : d[x] if isinstance(x,str) else x for x in ob[key]}
                ob[key] = newlst


In [292]:
class Outer(WarnOnExtraFieldsModel):
    A: List[At]
    B: List[Bt]
    C: List[Ct]
    D: List[Dt]

    @field_validator('B', mode='before')
    @classmethod
    def connect_B(cls, B, info: ValidationInfo):
        hydrate(B, info.data['A'], 'As')
        return B
    
    @field_validator('C', mode='before')
    @classmethod
    def connect_C(cls, C, info: ValidationInfo):
        hydrate(C, info.data['A'], 'As')
        hydrate(C, info.data['B'], 'Bs')
        return C
        
    @field_validator('D', mode='before')
    @classmethod
    def connect_D(cls, D, info: ValidationInfo):
        hydrate(D, info.data['C'], 'Cs')
        return D

In [293]:
pick = lambda L, size=3: {l.id: l for l in np.random.choice(L, size=size)}

In [295]:
A = [At(x=i, id='A' + str(i)) for i in range(10)]
B = [Bt(x=i, id='B' + str(i), As=pick(A,3)) for i in range(10)]
C = [Ct(x=i, id='C' + str(i), As=pick(A,3), Bs=pick(B,3)) for i in range(10)]
D = [Dt(x=i, id='D' + str(i), Cs=pick(C,3)) for i in range(5)]

In [297]:
O = Outer(A=A, B=B,C=C,D=D)
O2 = Outer(**json.loads(O.model_dump_json()))

O2 == O

True

In [298]:
O

Outer(A=[At(id='A0', x=0), At(id='A1', x=1), At(id='A2', x=2), At(id='A3', x=3), At(id='A4', x=4), At(id='A5', x=5), At(id='A6', x=6), At(id='A7', x=7), At(id='A8', x=8), At(id='A9', x=9)], B=[Bt(id='B0', x=0, As={'A5': At(id='A5', x=5)}), Bt(id='B1', x=1, As={'A2': At(id='A2', x=2), 'A4': At(id='A4', x=4), 'A6': At(id='A6', x=6)}), Bt(id='B2', x=2, As={'A3': At(id='A3', x=3), 'A4': At(id='A4', x=4)}), Bt(id='B3', x=3, As={'A4': At(id='A4', x=4), 'A7': At(id='A7', x=7), 'A8': At(id='A8', x=8)}), Bt(id='B4', x=4, As={'A5': At(id='A5', x=5), 'A4': At(id='A4', x=4), 'A0': At(id='A0', x=0)}), Bt(id='B5', x=5, As={'A6': At(id='A6', x=6), 'A7': At(id='A7', x=7), 'A2': At(id='A2', x=2)}), Bt(id='B6', x=6, As={'A3': At(id='A3', x=3), 'A1': At(id='A1', x=1)}), Bt(id='B7', x=7, As={'A2': At(id='A2', x=2), 'A1': At(id='A1', x=1)}), Bt(id='B8', x=8, As={'A1': At(id='A1', x=1), 'A8': At(id='A8', x=8), 'A7': At(id='A7', x=7)}), Bt(id='B9', x=9, As={'A3': At(id='A3', x=3), 'A1': At(id='A1', x=1), 'A2

In [300]:
print(O.model_dump_json(indent=2))

{
  "A": [
    {
      "id": "A0",
      "x": 0
    },
    {
      "id": "A1",
      "x": 1
    },
    {
      "id": "A2",
      "x": 2
    },
    {
      "id": "A3",
      "x": 3
    },
    {
      "id": "A4",
      "x": 4
    },
    {
      "id": "A5",
      "x": 5
    },
    {
      "id": "A6",
      "x": 6
    },
    {
      "id": "A7",
      "x": 7
    },
    {
      "id": "A8",
      "x": 8
    },
    {
      "id": "A9",
      "x": 9
    }
  ],
  "B": [
    {
      "id": "B0",
      "x": 0,
      "As": [
        "A5"
      ]
    },
    {
      "id": "B1",
      "x": 1,
      "As": [
        "A2",
        "A4",
        "A6"
      ]
    },
    {
      "id": "B2",
      "x": 2,
      "As": [
        "A3",
        "A4"
      ]
    },
    {
      "id": "B3",
      "x": 3,
      "As": [
        "A4",
        "A7",
        "A8"
      ]
    },
    {
      "id": "B4",
      "x": 4,
      "As": [
        "A5",
        "A4",
        "A0"
      ]
    },
    {
      "id": "B5",
      "x": 5,
